# Environment Setup

In [ ]:
#install packages
!pip install datasets -qq
!pip install evaluate -qq
!pip install transformers -qq
!pip install huggingface_hub -qq
!pip install neural-compressor
!pip install torchprofile
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests, zipfile, io
import time
import torch
import torchvision.transforms as transforms

from datasets import load_dataset
from PIL import Image
from torch import nn, optim
from torchvision import datasets, transforms
from torch.nn import functional as F
from transformers import AutoImageProcessor, AutoModelForImageClassification
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from torch.utils.data import DataLoader
#from torchprofile import profile_macs

# Importing Model and Processor

In [ ]:
# Model
model_name = 'microsoft/swinv2-tiny-patch4-window16-256'
model = AutoModelForImageClassification.from_pretrained(model_name)

# Processor
processor_name = 'microsoft/swinv2-tiny-patch4-window16-256'
processor = AutoImageProcessor.from_pretrained(processor_name)

In [ ]:
# copy the original model
from copy import deepcopy
original_model = deepcopy(model)

In [ ]:
model.state_dict().keys()

odict_keys(['swinv2.embeddings.patch_embeddings.projection.weight', 'swinv2.embeddings.patch_embeddings.projection.bias', 'swinv2.embeddings.norm.weight', 'swinv2.embeddings.norm.bias', 'swinv2.encoder.layers.0.blocks.0.attention.self.logit_scale', 'swinv2.encoder.layers.0.blocks.0.attention.self.continuous_position_bias_mlp.0.weight', 'swinv2.encoder.layers.0.blocks.0.attention.self.continuous_position_bias_mlp.0.bias', 'swinv2.encoder.layers.0.blocks.0.attention.self.continuous_position_bias_mlp.2.weight', 'swinv2.encoder.layers.0.blocks.0.attention.self.query.weight', 'swinv2.encoder.layers.0.blocks.0.attention.self.query.bias', 'swinv2.encoder.layers.0.blocks.0.attention.self.key.weight', 'swinv2.encoder.layers.0.blocks.0.attention.self.value.weight', 'swinv2.encoder.layers.0.blocks.0.attention.self.value.bias', 'swinv2.encoder.layers.0.blocks.0.attention.output.dense.weight', 'swinv2.encoder.layers.0.blocks.0.attention.output.dense.bias', 'swinv2.encoder.layers.0.blocks.0.layernor

In [ ]:
model

Swinv2ForImageClassification(
  (swinv2): Swinv2Model(
    (embeddings): Swinv2Embeddings(
      (patch_embeddings): Swinv2PatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swinv2Encoder(
      (layers): ModuleList(
        (0): Swinv2Stage(
          (blocks): ModuleList(
            (0-1): 2 x Swinv2Layer(
              (attention): Swinv2Attention(
                (self): Swinv2SelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=3, bias=False)
                  )
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96

# Loading Imagenet Dataset

In [ ]:
 from huggingface_hub import login
 login(token="your token")

In [ ]:
from datasets import load_dataset
# Load dataset
dataset1 = load_dataset("imagenet-1k", split="validation", trust_remote_code=True, streaming=True)
dataset2 = load_dataset("imagenet-1k", split="train", trust_remote_code=True, streaming=True)

In [ ]:
train_data = list(dataset2.take(1000))

In [ ]:
val_data = list(dataset1.take(1000))

## Building Data Loader

In [ ]:
from torch.utils.data import DataLoader, Dataset
# Define a custom dataset class using the processor
class ImageNetDataset(Dataset):
    def __init__(self, data, processor):
        self.data = data
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = item['image']

        # Convert grayscale image to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')

        label = item['label']
        # Process image using the processor
        processed_image = self.processor(images=image, return_tensors="pt").pixel_values
        return processed_image.squeeze(), label



# Create the dataset
val_dataset = ImageNetDataset(val_data, processor)
train_dataset = ImageNetDataset(train_data, processor)

# Create DataLoader with batch size 2
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)


# Torch Pruning API

## L1 Unstructured Pruning

In [ ]:
model.swinv2.encoder.layers[3].blocks[1].layernorm_after

LayerNorm((768,), eps=1e-05, elementwise_affine=True)

In [ ]:
parameters_to_prune = (
    #(model.swinv2.encoder.layers[3].blocks[1].output.dense, 'weight'),
    #(model.swinv2.encoder.layers[3].blocks[1].output.dense, 'bias'),
    #(model.swinv2.encoder.layers[3].blocks[1].layernorm_after, 'weight'),
    #(model.swinv2.encoder.layers[3].blocks[1].layernorm_after, 'bias'),

    (model.swinv2.layernorm, 'weight'),
    (model.swinv2.layernorm, 'bias'),

)


prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.5,
)

### Sparsity Check

In [ ]:
'''
print(
    "Sparsity in layer[3].blocks[1].output.dense.weight: {:.2f}%".format(
        100. * float(torch.sum(model.swinv2.encoder.layers[3].blocks[1].output.dense.weight == 0))
        / float(model.swinv2.encoder.layers[3].blocks[1].output.dense.weight.nelement())
    )
)

print(
    "Sparsity in layer[3].blocks[1].output.dense.bias: {:.2f}%".format(
        100. * float(torch.sum(model.swinv2.encoder.layers[3].blocks[1].output.dense.bias == 0))
        / float(model.swinv2.encoder.layers[3].blocks[1].output.dense.bias.nelement())
    )
)
print(
    "Sparsity in layer[3].blocks[1].layernorm_after.weight: {:.2f}%".format(
        100. * float(torch.sum(model.swinv2.encoder.layers[3].blocks[1].layernorm_after.weight == 0))
        / float(model.swinv2.encoder.layers[3].blocks[1].layernorm_after.weight.nelement())
    )
)

print(
    "Sparsity in layer[3].blocks[1].layernorm_after.bias: {:.2f}%".format(
        100. * float(torch.sum(model.swinv2.encoder.layers[3].blocks[1].layernorm_after.bias == 0))
        / float(model.swinv2.encoder.layers[3].blocks[1].layernorm_after.bias.nelement())
    )
)

'''

print(
    "Sparsity in layernorm.weight: {:.2f}%".format(
        100. * float(torch.sum(model.swinv2.layernorm.weight == 0))
        / float(model.swinv2.layernorm.weight.nelement())
    )
)

print(
    "Sparsity in layernorm.bias: {:.2f}%".format(
        100. * float(torch.sum(model.swinv2.layernorm.bias == 0))
        / float(model.swinv2.layernorm.bias.nelement())
    )
)

Sparsity in layernorm.weight: 0.52%
Sparsity in layernorm.bias: 99.48%


In [ ]:
# Remove pruning re-parametrization
#prune.remove(model.swinv2.encoder.layers[3].blocks[1].output.dense, 'weight')
#prune.remove(model.swinv2.encoder.layers[3].blocks[1].output.dense, 'bias')
#prune.remove(model.swinv2.encoder.layers[3].blocks[1].layernorm_after, 'weight')
#prune.remove(model.swinv2.encoder.layers[3].blocks[1].layernorm_after, 'bias')

prune.remove(model.swinv2.layernorm, 'weight')
prune.remove(model.swinv2.layernorm, 'bias')

# Save the model
torch.save(model.state_dict(), 'pruned_model.pth')

In [ ]:
# Load the pruned model
model.load_state_dict(torch.load('pruned_model.pth'))

<All keys matched successfully>

### Fine-tuning The Model

In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler
from PIL import Image
import time

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

# Fine-tuning settings
learning_rate = 1e-5  # Low learning rate
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1500 [00:00<?, ?it/s]

### Evaluating The Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model.to(device)

Swinv2ForImageClassification(
  (swinv2): Swinv2Model(
    (embeddings): Swinv2Embeddings(
      (patch_embeddings): Swinv2PatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swinv2Encoder(
      (layers): ModuleList(
        (0): Swinv2Stage(
          (blocks): ModuleList(
            (0-1): 2 x Swinv2Layer(
              (attention): Swinv2Attention(
                (self): Swinv2SelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=3, bias=False)
                  )
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96

In [ ]:
original_model.to(device)

Swinv2ForImageClassification(
  (swinv2): Swinv2Model(
    (embeddings): Swinv2Embeddings(
      (patch_embeddings): Swinv2PatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swinv2Encoder(
      (layers): ModuleList(
        (0): Swinv2Stage(
          (blocks): ModuleList(
            (0-1): 2 x Swinv2Layer(
              (attention): Swinv2Attention(
                (self): Swinv2SelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=3, bias=False)
                  )
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96

In [ ]:
import evaluate
import time
import numpy as np

def evaluate_model(model, eval_dataloader):
    # Load the accuracy metric
    # Load metrics
    accuracy_metric = evaluate.load("accuracy")
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")
    f1_metric = evaluate.load("f1")

    # Set the model to evaluation mode
    model.eval()

    inference_times = []
    flops_list = []
    macs_list = []

    # Evaluate the model
    for images, labels in eval_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        with torch.no_grad():
          start_time = time.time()  # Start time for inference
          outputs = model(images)
          end_time = time.time()  # End time for inference
          inference_times.append(end_time - start_time)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        accuracy_metric.add_batch(predictions=predictions, references=labels)
        precision_metric.add_batch(predictions=predictions, references=labels)
        recall_metric.add_batch(predictions=predictions, references=labels)
        f1_metric.add_batch(predictions=predictions, references=labels)

    # Compute accuracy, precision, recall, and F1
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')

    # Compute average inference, average_flops, average_macs time per batch
    avg_inference_time = sum(inference_times) / len(inference_times)

    return accuracy, precision, recall, f1, avg_inference_time

#### Accuracy, Precision, Recall, F1 Score

In [ ]:
accuracy, precision, recall, f1, avg_inference_time = evaluate_model(model, val_loader)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
orig_accuracy, orig_precision, orig_recall, orig_f1, orig_avg_inference_time = evaluate_model(original_model, val_loader)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### NPs, MACs, FLOPs

In [ ]:
from torchprofile import profile_macs
from fvcore.nn import FlopCountAnalysis, flop_count_table

image = val_data[0]['image']

# Define a input tensor
input_tensor = processor(images = image, return_tensors = 'pt')['pixel_values'].to(device)


In [ ]:
# Pruned Model

# Calculate the NPs (Number of Parameters)
#pru_total_params = sum(p.numel() for p in model.parameters())

# Count the number of zeroed parameters
zero_params = sum(torch.sum(p == 0).item() for p in model.parameters())
print(f"Number of zeroed parameters after pruning: {zero_params}")

pru_total_params = org_total_params - zero_params

# Calculate MACs (Multiply-Accumulate Operations)
pru_macs = profile_macs(model, input_tensor)

# Calculate FLOPs
pru_flops = FlopCountAnalysis(model, input_tensor)


Number of zeroed parameters after pruning: 768


/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::remainder". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::pad". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::permute". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::linalg_vector_norm". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::clamp_min". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profil

In [ ]:
# Original Model

# Calculate the NPs (Number of Parameters)
org_total_params = sum(p.numel() for p in original_model.parameters())

# Calculate MACs (Multiply-Accumulate Operations)
org_macs = profile_macs(original_model, input_tensor)

# Calculate FLOPs
org_flops = FlopCountAnalysis(model, input_tensor)


/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::remainder". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::pad". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::permute". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::linalg_vector_norm". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::clamp_min". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profil

#### Compression Rate

In [ ]:
# Compression Rate
compression_rate = org_total_params/ pru_total_params

#### Priniting The Results

In [ ]:
# Print Pruned Model Results
print('---- Pruned Model Results ---- \n')
print(f'Accuracy: {accuracy["accuracy"] * 100:.2f}% \n')
print(f'Precision: {precision["precision"] * 100:.2f}% \n')
print(f'Recall: {recall["recall"] * 100:.2f}% \n')
print(f'F1 Score: {f1["f1"] * 100:.2f}% \n')
print(f'Average Inference Time per Batch: {avg_inference_time:.6f} seconds \n')

print('Calculating NPs, FLOPs, MACs for single sample \n')
print(f'Number of Non Zeroed Parameters (NPs): {pru_total_params / 1e6:.8f} M \n')
print(f"Number of Zeroed Parameters After Pruning: {zero_params} \n")
print(f'FLOPs: {flop_count_table(pru_flops)} \n')
print(f'MACs: {pru_macs / 1e9:.2f} G\n')


---- Pruned Model Results ---- 

Accuracy: 81.64% 

Precision: 85.02% 

Recall: 81.64% 

F1 Score: 81.54% 

Average Inference Time per Batch: 0.988347 seconds 

Calculating NPs, FLOPs, MACs for single sample 

Number of Non Zeroed Parameters (NPs): 28.34638600 M 

Number of Zeroed Parameters After Pruning: 768 

FLOPs: | module                                           | #parameters or shape   | #flops     |
|:-------------------------------------------------|:-----------------------|:-----------|
| model                                            | 28.347M                | 6.683G     |
|  swinv2                                          |  27.578M               |  6.682G    |
|   swinv2.embeddings                              |   4.896K               |   20.84M   |
|    swinv2.embeddings.patch_embeddings.projection |    4.704K              |    18.874M |
|    swinv2.embeddings.norm                        |    0.192K              |    1.966M  |
|   swinv2.encoder.layers                 

In [ ]:
# Print Original Model Results
print('---- Original Model Results ---- \n')
print(f'Accuracy: {orig_accuracy["accuracy"] * 100:.2f}% \n')
print(f'Precision: {orig_precision["precision"] * 100:.2f}% \n')
print(f'Recall: {orig_recall["recall"] * 100:.2f}% \n')
print(f'F1 Score: {orig_f1["f1"] * 100:.2f}% \n')
print(f'Average Inference Time per Batch: {orig_avg_inference_time:.6f} seconds \n')

print('Calculating NPs, FLOPs, MACs for single sample \n')
print(f'Number of Parameters (NPs): {org_total_params / 1e6:.8f} M \n')
print(f'FLOPs: {flop_count_table(org_flops)} \n')
print(f'MACs: {org_macs / 1e9:.2f} G\n')


---- Original Model Results ---- 

Accuracy: 81.84% 

Precision: 85.03% 

Recall: 81.84% 

F1 Score: 81.70% 

Average Inference Time per Batch: 1.126204 seconds 

Calculating NPs, FLOPs, MACs for single sample 

Number of Parameters (NPs): 28.34715400 M 

FLOPs: | module                                           | #parameters or shape   | #flops     |
|:-------------------------------------------------|:-----------------------|:-----------|
| model                                            | 28.347M                | 6.683G     |
|  swinv2                                          |  27.578M               |  6.682G    |
|   swinv2.embeddings                              |   4.896K               |   20.84M   |
|    swinv2.embeddings.patch_embeddings.projection |    4.704K              |    18.874M |
|    swinv2.embeddings.norm                        |    0.192K              |    1.966M  |
|   swinv2.encoder.layers                          |   27.572M              |   6.661G   |
|    swin

In [ ]:
print('Calculating Compression Rate for single sample \n')
print(f'Compression Rate: {compression_rate} \n'

# Intel Neural Compressor Pruning API

In [ ]:
!pip install optimum[intel]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.7/424.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/224.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.0 MB/s eta 0:00:00


In [ ]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 19.5 MB/s eta 0:00:00


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from neural_compressor.training import prepare_compression, WeightPruningConfig
from transformers.data.data_collator import default_data_collator
from optimum.intel.neural_compressor import INCTrainer

## Previous Tried Method (Wrong Way)

In [ ]:
# Pruning configuration
pruning_config = {
    "pruning_type": "snip_momentum",
    "pruning_scope": "global",
    "sparsity_decay_type": "exp",
    "op_names": ["layer"],
    "excluded_op_names": ["head", "fc", ".downsample"],
    "pruning_op_types": ["Conv2d"],
    "max_sparsity_ratio_per_op": 0.98
}

In [ ]:
from torch.utils.data import DataLoader, Dataset
# Define a custom dataset class using the processor
class ImageNetDataset(Dataset):
    def __init__(self, data, processor):
        self.data = data
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = item['image']

        # Convert grayscale image to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')

        label = item['label']
        # Process image using the processor
        processed_image = self.processor(images=image, return_tensors="pt").pixel_values
        return {"pixel_values": processed_image.squeeze(), "labels": label}



# Create the dataset
val_dataset = ImageNetDataset(val_data, processor)
train_dataset = ImageNetDataset(train_data, processor)


In [ ]:
# Setup evaluation
import evaluate
import numpy as np
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Load metrics
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

In [ ]:
original_NPs = sum(p.numel() for p in original_model.parameters())
print(f"Original Model Number of Parameters: {original_NPs / 1e6:.2f} M")

Original Model Number of Parameters: 28.35 M


In [ ]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torchprofile import profile_macs

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)

    # Compute precision, recall, and F1
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')

    # Compute inference time and FLOPs for multiple samples
    num_samples = 100  # Number of samples to average over
    inference_times = []
    flops_list = []
    macs_list = []

    for i in range(num_samples):
        sample_inputs = torch.tensor(logits[i:i+1])  # Using a single input from logits

        # Compute inference time
        start_time = time.time()
        with torch.no_grad():
            _ = model(sample_inputs)
        inference_time = time.time() - start_time
        inference_times.append(inference_time)

        # Compute FLOPs
        flops = torchprofile.profile_macs(model, sample_inputs)
        flops_list.append(flops)

        # Compute MACs
        macs = torchprofile.profile_macs(model, sample_inputs)
        macs_list.append(macs)

    avg_inference_time = sum(inference_times) / num_samples
    avg_flops = sum(flops_list) / num_samples
    avg_macs = sum(macs_list) / num_samples

    # Compute compression rate (requires original model size and compressed model size)
    original_model_size = original_NPs
    compressed_model_size = sum(p.numel() for p in model.parameters())
    compression_rate = original_model_size / compressed_model_size

    return {
        'Accuracy': accuracy['accuracy'],
        'Precision': precision['precision'],
        'Recall': recall['recall'],
        'F1': f1['f1'],
        'Inference_Time': avg_inference_time,
        'Flops': avg_flops,
        'MACs': avg_macs,
        'NPs': compressed_model_size,
        'Compression_Rate': compression_rate
    }

In [ ]:
# Create pruning config and prepare compression
num_iterations = int(len(train_dataset)/2)
start_step = int(0.1 * num_iterations)  # Example warmup period
end_step = int(0.9 * num_iterations)  # Example cooldown period

configs = WeightPruningConfig(
    pruning_configs=[pruning_config],
    target_sparsity=0.5,
    pattern='4x1',
    pruning_frequency=100,
    start_step=start_step,
    end_step=end_step
)

In [ ]:
print(num_iterations)
print(start_step)
print(end_step)

500
50
450


In [ ]:
# Training arguments
save_dir = "prunedModel"

training_args = TrainingArguments(
    save_dir,
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.001,
    metric_for_best_model="f1",
    greater_is_better=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
train_loader

In [ ]:
# Initialize the trainer with INCTrainer
trainer = INCTrainer(
    model=model,
    pruning_config=pruning_config,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #compute_metrics=compute_metrics,
    data_collator=default_data_collator,
    tokenizer=processor
)


In [ ]:
# Train and prune iteratively
for epoch in range(int(training_args.num_train_epochs)):
    train_result = trainer.train()
    trainer.save_model(f"{save_dir}/epoch_{epoch}")
    optimized_model = AutoModelForImageClassification.from_pretrained(f"{save_dir}/epoch_{epoch}")
    memory_footprint = optimized_model.get_memory_footprint()
    print(f"Model memory footprint after epoch {epoch}: {memory_footprint} bytes")

***** Running training *****
  Num examples = 1,000
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 500
  Number of trainable parameters = 28,347,154


Epoch,Training Loss,Validation Loss


Saving model checkpoint to prunedModel/checkpoint-500
Configuration saved in prunedModel/checkpoint-500/inc_config.json
Model weights saved in prunedModel/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: not enough values to unpack (expected 4, got 2)

## Correct Way of Using Neural Compressor Pruning API

### Functions and Confugurations

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model.to(device)

Swinv2ForImageClassification(
  (swinv2): Swinv2Model(
    (embeddings): Swinv2Embeddings(
      (patch_embeddings): Swinv2PatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swinv2Encoder(
      (layers): ModuleList(
        (0): Swinv2Stage(
          (blocks): ModuleList(
            (0-1): 2 x Swinv2Layer(
              (attention): Swinv2Attention(
                (self): Swinv2SelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=3, bias=False)
                  )
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96

In [ ]:
original_model.to(device)

Swinv2ForImageClassification(
  (swinv2): Swinv2Model(
    (embeddings): Swinv2Embeddings(
      (patch_embeddings): Swinv2PatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swinv2Encoder(
      (layers): ModuleList(
        (0): Swinv2Stage(
          (blocks): ModuleList(
            (0-1): 2 x Swinv2Layer(
              (attention): Swinv2Attention(
                (self): Swinv2SelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=3, bias=False)
                  )
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96

#### Configuration Setup

In [ ]:
# Pruning configuration
pruning_config = [
    {
    "target_sparsity": 0.9,  # Target sparsity ratio of modules.
    "pruning_type": "snip_momentum", # Default pruning type.
    "pattern": "4x1",  # Default pruning pattern.
    "pruning_scope": "global",  # Default pruning scope.
    "sparsity_decay_type": "exp",  # Function applied to control pruning rate.
    "op_names": ["layernorm"],  # A list of modules that would be pruned.
    "excluded_op_names": ["layernorm_before", "layernorm_after"],  # A list of modules that would not be pruned.
    "pruning_op_types": ["LayerNorm"],  # Default pruning operation types.
    "start_step": 100,  # Step at which to begin pruning.
    "end_step": 400,  # Step at which to end pruning.
    "pruning_frequency": 75,  # Frequency of applying pruning, The recommended setting is one fortieth of the pruning steps.
    "min_sparsity_ratio_per_op": 0.0,  # Minimum sparsity ratio of each module.
    "max_sparsity_ratio_per_op": 0.98,  # Maximum sparsity ratio of each module.
}
]


#### Training Setup

In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler
from PIL import Image
import time

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
from transformers import get_scheduler

# Fine-tuning settings
learning_rate = 1e-5  # Low learning rate
num_train_epochs = 1
num_training_steps = num_train_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
num_training_steps

500

#### Evaluation Function Setup

In [ ]:
import evaluate
import time
import numpy as np

def evaluate_model(model, eval_dataloader):
    # Load the accuracy metric
    # Load metrics
    accuracy_metric = evaluate.load("accuracy")
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")
    f1_metric = evaluate.load("f1")

    # Set the model to evaluation mode
    model.eval()

    inference_times = []
    flops_list = []
    macs_list = []

    # Evaluate the model
    for images, labels in eval_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        with torch.no_grad():
          start_time = time.time()  # Start time for inference
          outputs = model(images)
          end_time = time.time()  # End time for inference
          inference_times.append(end_time - start_time)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        accuracy_metric.add_batch(predictions=predictions, references=labels)
        precision_metric.add_batch(predictions=predictions, references=labels)
        recall_metric.add_batch(predictions=predictions, references=labels)
        f1_metric.add_batch(predictions=predictions, references=labels)

    # Compute accuracy, precision, recall, and F1
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')

    # Compute average inference, average_flops, average_macs time per batch
    avg_inference_time = sum(inference_times) / len(inference_times)

    return accuracy, precision, recall, f1, avg_inference_time

In [ ]:
from torchprofile import profile_macs
from fvcore.nn import FlopCountAnalysis, flop_count_table

image = val_data[0]['image']

# Define a input tensor
input_tensor = processor(images = image, return_tensors = 'pt')['pixel_values'].to(device)


### Original Model Evaluation

In [ ]:
original_model.to(device)

Swinv2ForImageClassification(
  (swinv2): Swinv2Model(
    (embeddings): Swinv2Embeddings(
      (patch_embeddings): Swinv2PatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swinv2Encoder(
      (layers): ModuleList(
        (0): Swinv2Stage(
          (blocks): ModuleList(
            (0-1): 2 x Swinv2Layer(
              (attention): Swinv2Attention(
                (self): Swinv2SelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=3, bias=False)
                  )
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96

##### Accuracy, Precision, Recall, F1 Score

In [ ]:
orig_accuracy, orig_precision, orig_recall, orig_f1, orig_avg_inference_time = evaluate_model(model, val_loader)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### NPs, MACs, FLOPs

In [ ]:
# Original Model Evaluate

# Calculate the NPs (Number of Parameters)
org_total_params = sum(p.numel() for p in original_model.parameters())

# Calculate MACs (Multiply-Accumulate Operations)
org_macs = profile_macs(original_model, input_tensor)

# Calculate FLOPs
org_flops = FlopCountAnalysis(model, input_tensor)


/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::remainder". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::pad". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::permute". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::linalg_vector_norm". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::clamp_min". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profil

In [ ]:
# Print Original Model Results
print('---- Original Model Results ---- \n')
print(f'Accuracy: {orig_accuracy["accuracy"] * 100:.2f}% \n')
print(f'Precision: {orig_precision["precision"] * 100:.2f}% \n')
print(f'Recall: {orig_recall["recall"] * 100:.2f}% \n')
print(f'F1 Score: {orig_f1["f1"] * 100:.2f}% \n')
print(f'Average Inference Time per Batch: {orig_avg_inference_time:.6f} seconds \n')

print('Calculating NPs, FLOPs, MACs for single sample \n')
print(f'Number of Parameters (NPs): {org_total_params / 1e6:.2f} M \n')
print(f'FLOPs: {flop_count_table(org_flops)} \n')
print(f'MACs: {org_macs / 1e9:.2f} G\n')


---- Original Model Results ---- 

Accuracy: 81.84% 

Precision: 85.03% 

Recall: 81.84% 

F1 Score: 81.70% 

Average Inference Time per Batch: 0.033737 seconds 

Calculating NPs, FLOPs, MACs for single sample 

Number of Parameters (NPs): 28.35 M 

FLOPs: | module                                           | #parameters or shape   | #flops     |
|:-------------------------------------------------|:-----------------------|:-----------|
| model                                            | 28.347M                | 6.683G     |
|  swinv2                                          |  27.578M               |  6.682G    |
|   swinv2.embeddings                              |   4.896K               |   20.84M   |
|    swinv2.embeddings.patch_embeddings.projection |    4.704K              |    18.874M |
|    swinv2.embeddings.norm                        |    0.192K              |    1.966M  |
|   swinv2.encoder.layers                          |   27.572M              |   6.661G   |
|    swinv2.enc

### Training Aware Iterative Magnitude Pruning (IMP)

#### IMP Training

In [ ]:
from neural_compressor.training import prepare_compression, WeightPruningConfig

config = WeightPruningConfig(pruning_config)
compression_manager = prepare_compression(model, config)  # Define a pruning object.
compression_manager.callbacks.on_train_begin()  ## insert hook

for epoch in range(num_train_epochs):
    model.train()
    for step, batch in enumerate(train_loader):
        batch = [b.to(device) for b in batch]
        compression_manager.callbacks.on_step_begin(step)
        images, labels = batch
        outputs = model(images, labels=labels)
        loss = outputs.loss
        loss.backward()
        compression_manager.callbacks.on_before_optimizer_step()
        optimizer.step()
        compression_manager.callbacks.on_after_optimizer_step()
        lr_scheduler.step()
        model.zero_grad()
compression_manager.callbacks.on_train_end()


2024-07-22 19:49:42 [INFO] {
2024-07-22 19:49:42 [INFO]     'target_sparsity': 0.9,
2024-07-22 19:49:42 [INFO]     'pruning_type': 'magnitude',
2024-07-22 19:49:42 [INFO]     'pattern': '4x1',
2024-07-22 19:49:42 [INFO]     'pruning_scope': 'global',
2024-07-22 19:49:42 [INFO]     'sparsity_decay_type': 'exp',
2024-07-22 19:49:42 [INFO]     'op_names': [
2024-07-22 19:49:42 [INFO]         'layernorm'
2024-07-22 19:49:42 [INFO]     ],
2024-07-22 19:49:42 [INFO]     'excluded_op_names': [
2024-07-22 19:49:42 [INFO]         'layernorm_before',
2024-07-22 19:49:42 [INFO]         'layernorm_after',
2024-07-22 19:49:42 [INFO]         'classifier'
2024-07-22 19:49:42 [INFO]     ],
2024-07-22 19:49:42 [INFO]     'pruning_op_types': [
2024-07-22 19:49:42 [INFO]         'LayerNorm'
2024-07-22 19:49:42 [INFO]     ],
2024-07-22 19:49:42 [INFO]     'start_step': 100,
2024-07-22 19:49:42 [INFO]     'end_step': 400,
2024-07-22 19:49:42 [INFO]     'pruning_frequency': 75,
2024-07-22 19:49:42 [INFO]   

#### Save and Load the Model

In [ ]:
# Save the pruned model
torch.save(model.state_dict(), 'pruned_model.pth')

In [ ]:
# load the model
#pruned_model.load_state_dict(torch.load('pruned_model.pth'))

#### Evaluate The Model

In [ ]:
model.to(device)

Swinv2ForImageClassification(
  (swinv2): Swinv2Model(
    (embeddings): Swinv2Embeddings(
      (patch_embeddings): Swinv2PatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swinv2Encoder(
      (layers): ModuleList(
        (0): Swinv2Stage(
          (blocks): ModuleList(
            (0-1): 2 x Swinv2Layer(
              (attention): Swinv2Attention(
                (self): Swinv2SelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=3, bias=False)
                  )
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96

#### Accuracy, Precision, Recall, F1 Score

In [ ]:
accuracy, precision, recall, f1, avg_inference_time = evaluate_model(model, val_loader)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### NPs, MACs, FLOPs

In [ ]:
# Pruned Model

# Calculate the NPs (Number of Parameters)
pru_total_params = sum(p.numel() for p in model.parameters())

# Calculate MACs (Multiply-Accumulate Operations)
pru_macs = profile_macs(model, input_tensor)

# Calculate FLOPs
pru_flops = FlopCountAnalysis(model, input_tensor)


/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::remainder". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::pad". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::permute". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::linalg_vector_norm". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::clamp_min". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profil

#### Compression Rate

In [ ]:
# Compression Rate
compression_rate = org_total_params/ pru_total_params

#### Printing The Results

In [ ]:
# Print Pruned Model Results
print('---- Pruned Model Results ---- \n')
print(f'Accuracy: {accuracy["accuracy"] * 100:.2f}% \n')
print(f'Precision: {precision["precision"] * 100:.2f}% \n')
print(f'Recall: {recall["recall"] * 100:.2f}% \n')
print(f'F1 Score: {f1["f1"] * 100:.2f}% \n')
print(f'Average Inference Time per Batch: {avg_inference_time:.6f} seconds \n')

print('Calculating NPs, FLOPs, MACs for single sample \n')
print(f'Number of Parameters (NPs): {pru_total_params / 1e6:.2f} M \n')
print(f'FLOPs: {flop_count_table(pru_flops)} \n')
print(f'MACs: {pru_macs / 1e9:.2f} G\n')



# Print Original Model Results
print('---- Original Model Results ---- \n')
print(f'Accuracy: {orig_accuracy["accuracy"] * 100:.2f}% \n')
print(f'Precision: {orig_precision["precision"] * 100:.2f}% \n')
print(f'Recall: {orig_recall["recall"] * 100:.2f}% \n')
print(f'F1 Score: {orig_f1["f1"] * 100:.2f}% \n')
print(f'Average Inference Time per Batch: {orig_avg_inference_time:.6f} seconds \n')

print('Calculating NPs, FLOPs, MACs for single sample \n')
print(f'Number of Parameters (NPs): {org_total_params / 1e6:.2f} M \n')
print(f'FLOPs: {flop_count_table(org_flops)} \n')
print(f'MACs: {org_macs / 1e9:.2f} G\n')


print('Calculating Compression Rate for single sample \n')
print(f'Compression Rate: {compression_rate} \n')

---- Pruned Model Results ---- 

Accuracy: 68.86% 

Precision: 73.15% 

Recall: 68.86% 

F1 Score: 68.52% 

Average Inference Time per Batch: 0.032662 seconds 

Calculating NPs, FLOPs, MACs for single sample 

Number of Parameters (NPs): 28.35 M 

FLOPs: | module                                           | #parameters or shape   | #flops     |
|:-------------------------------------------------|:-----------------------|:-----------|
| model                                            | 28.347M                | 6.683G     |
|  swinv2                                          |  27.578M               |  6.682G    |
|   swinv2.embeddings                              |   4.896K               |   20.84M   |
|    swinv2.embeddings.patch_embeddings.projection |    4.704K              |    18.874M |
|    swinv2.embeddings.norm                        |    0.192K              |    1.966M  |
|   swinv2.encoder.layers                          |   27.572M              |   6.661G   |
|    swinv2.encod

In [ ]:
model.swinv2.layernorm.weight


Parameter containing:
tensor([0.0000, 0.0000, 3.9999, 0.0000, 0.0000, 3.6371, 0.0000, 3.2159, 0.0000,
        0.0000, 2.7580, 0.0000, 0.0000, 2.7334, 0.0000, 2.9794, 0.0000, 0.0000,
        3.1637, 0.0000, 0.0000, 3.1344, 0.0000, 3.0016, 0.0000, 0.0000, 3.3643,
        0.0000, 0.0000, 2.7287, 0.0000, 3.0509, 0.0000, 2.2982, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.5868, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 3.9917, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 3.2445, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 3.0781, 3.3005, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 2.6215, 2.8647, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        3.1280, 3.0263, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 3.0420,
        2.9639, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 2.8755, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 2.

In [ ]:
original_model.swinv2.layernorm.weight

Parameter containing:
tensor([ 1.2858e+00,  2.7892e+00,  3.9985e+00,  2.7193e+00,  2.3325e+00,
         3.6349e+00,  2.9811e+00,  3.2132e+00,  2.7926e+00,  2.2835e+00,
         2.7539e+00,  2.9471e+00,  2.9324e+00,  2.7301e+00,  2.7966e+00,
         2.9769e+00,  2.6193e+00,  3.0922e+00,  3.1618e+00,  2.3429e+00,
         2.9898e+00,  3.1319e+00,  3.0971e+00,  2.9987e+00,  2.8678e+00,
         2.6130e+00,  3.3611e+00,  2.8634e+00,  2.6790e+00,  2.7263e+00,
         2.2420e+00,  3.0492e+00,  2.3291e+00,  2.2954e+00,  2.6931e+00,
         2.5969e+00,  1.5440e+00,  2.5684e+00,  2.6792e+00,  2.6313e+00,
         3.1546e+00,  2.5841e+00,  2.5663e+00,  3.8745e+00,  2.7666e+00,
         3.3117e+00,  2.9660e+00,  2.8197e+00,  2.6077e+00,  3.9885e+00,
         3.0107e+00,  1.2215e+00,  2.2473e+00,  2.8262e+00,  2.3194e+00,
         2.5544e+00,  2.6423e+00,  3.2415e+00,  2.8065e+00,  2.9361e+00,
         2.9766e+00,  2.5896e+00,  2.2447e+00,  3.1194e+00,  2.6064e+00,
         3.0744e+00,  3.2981e

In [ ]:
model.swinv2.layernorm.bias

Parameter containing:
tensor([ 4.0009e-01,  7.3587e-02,  2.7817e-02,  7.3353e-02, -3.0153e-02,
        -1.2344e-02,  1.2511e-01,  9.1488e-02,  9.7834e-02,  1.7401e-01,
        -2.6908e-02,  1.1648e-01,  3.8689e-02, -3.5162e-03,  6.0923e-02,
         1.9436e-01,  3.4060e-02, -1.6344e-02,  4.8195e-02,  1.3645e-01,
         5.3799e-02,  9.3810e-02,  1.1468e-02,  1.9894e-01,  4.4943e-02,
         1.6516e-01,  8.2310e-02,  1.4192e-01,  8.2171e-02,  1.3811e-01,
         2.4377e-01,  1.2331e-01,  1.2732e-01, -6.4108e-03,  1.5446e-01,
         2.8923e-02, -3.5759e-02, -1.5458e-02,  1.7139e-01,  9.0009e-02,
        -1.2834e-02,  1.3800e-01, -1.1105e-02,  1.3882e-01,  1.8741e-01,
         3.9892e-02, -2.4127e-02, -2.6214e-02,  1.2683e-01,  1.4315e-01,
         2.0022e-01, -2.2266e-01,  3.9629e-01,  2.2255e-01,  2.8936e-01,
         2.1067e-01,  8.2298e-02,  1.5629e-01,  1.3661e-01,  1.4859e-01,
         4.0312e-02,  2.0310e-01,  3.1857e-03,  1.0683e-01,  1.1829e-01,
         1.0917e-01,  8.0407e

### Training Aware SNIP Momentum

#### Snip Momentum Training

In [ ]:
from neural_compressor.training import prepare_compression, WeightPruningConfig

config = WeightPruningConfig(pruning_config)
compression_manager = prepare_compression(model, config)  # Define a pruning object.
compression_manager.callbacks.on_train_begin()  ## insert hook

for epoch in range(num_train_epochs):
    model.train()
    for step, batch in enumerate(train_loader):
        batch = [b.to(device) for b in batch]
        compression_manager.callbacks.on_step_begin(step)
        images, labels = batch
        outputs = model(images, labels=labels)
        loss = outputs.loss
        loss.backward()
        compression_manager.callbacks.on_before_optimizer_step()
        optimizer.step()
        compression_manager.callbacks.on_after_optimizer_step()
        lr_scheduler.step()
        model.zero_grad()
compression_manager.callbacks.on_train_end()


2024-07-22 19:30:03 [INFO] {
2024-07-22 19:30:03 [INFO]     'target_sparsity': 0.9,
2024-07-22 19:30:03 [INFO]     'pruning_type': 'snip_momentum',
2024-07-22 19:30:03 [INFO]     'pattern': '4x1',
2024-07-22 19:30:03 [INFO]     'pruning_scope': 'global',
2024-07-22 19:30:03 [INFO]     'sparsity_decay_type': 'exp',
2024-07-22 19:30:03 [INFO]     'op_names': [
2024-07-22 19:30:03 [INFO]         'layernorm'
2024-07-22 19:30:03 [INFO]     ],
2024-07-22 19:30:03 [INFO]     'excluded_op_names': [
2024-07-22 19:30:03 [INFO]         'layernorm_before',
2024-07-22 19:30:03 [INFO]         'layernorm_after',
2024-07-22 19:30:03 [INFO]         'classifier'
2024-07-22 19:30:03 [INFO]     ],
2024-07-22 19:30:03 [INFO]     'pruning_op_types': [
2024-07-22 19:30:03 [INFO]         'LayerNorm'
2024-07-22 19:30:03 [INFO]     ],
2024-07-22 19:30:03 [INFO]     'start_step': 100,
2024-07-22 19:30:03 [INFO]     'end_step': 400,
2024-07-22 19:30:03 [INFO]     'pruning_frequency': 75,
2024-07-22 19:30:03 [INFO

#### Save and Load the Model

In [ ]:
# Save the pruned model
torch.save(model.state_dict(), 'pruned_model.pth')

In [ ]:
# load the model
#pruned_model.load_state_dict(torch.load('pruned_model.pth'))

#### Evaluate The Model

In [ ]:
model.to(device)

Swinv2ForImageClassification(
  (swinv2): Swinv2Model(
    (embeddings): Swinv2Embeddings(
      (patch_embeddings): Swinv2PatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swinv2Encoder(
      (layers): ModuleList(
        (0): Swinv2Stage(
          (blocks): ModuleList(
            (0-1): 2 x Swinv2Layer(
              (attention): Swinv2Attention(
                (self): Swinv2SelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=3, bias=False)
                  )
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96

#### Accuracy, Precision, Recall, F1 Score

In [ ]:
accuracy, precision, recall, f1, avg_inference_time = evaluate_model(model, val_loader)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### NPs, MACs, FLOPs

In [ ]:
# Pruned Model

# Calculate the NPs (Number of Parameters)
pru_total_params = sum(p.numel() for p in model.parameters())

# Calculate MACs (Multiply-Accumulate Operations)
pru_macs = profile_macs(model, input_tensor)

# Calculate FLOPs
pru_flops = FlopCountAnalysis(model, input_tensor)


/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::remainder". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::pad". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::permute". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::linalg_vector_norm". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::clamp_min". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.10/dist-packages/torchprofile/profil

#### Compression Rate

In [ ]:
# Compression Rate
compression_rate = org_total_params/ pru_total_params

#### Printing The Results

In [ ]:
# Print Pruned Model Results
print('---- Pruned Model Results ---- \n')
print(f'Accuracy: {accuracy["accuracy"] * 100:.2f}% \n')
print(f'Precision: {precision["precision"] * 100:.2f}% \n')
print(f'Recall: {recall["recall"] * 100:.2f}% \n')
print(f'F1 Score: {f1["f1"] * 100:.2f}% \n')
print(f'Average Inference Time per Batch: {avg_inference_time:.6f} seconds \n')

print('Calculating NPs, FLOPs, MACs for single sample \n')
print(f'Number of Parameters (NPs): {pru_total_params / 1e6:.2f} M \n')
print(f'FLOPs: {flop_count_table(pru_flops)} \n')
print(f'MACs: {pru_macs / 1e9:.2f} G\n')


print('Calculating Compression Rate for single sample \n')
print(f'Compression Rate: {compression_rate} \n')

---- Pruned Model Results ---- 

Accuracy: 69.26% 

Precision: 74.95% 

Recall: 69.26% 

F1 Score: 68.99% 

Average Inference Time per Batch: 0.028742 seconds 

Calculating NPs, FLOPs, MACs for single sample 

Number of Parameters (NPs): 28.35 M 

FLOPs: | module                                           | #parameters or shape   | #flops     |
|:-------------------------------------------------|:-----------------------|:-----------|
| model                                            | 28.347M                | 6.683G     |
|  swinv2                                          |  27.578M               |  6.682G    |
|   swinv2.embeddings                              |   4.896K               |   20.84M   |
|    swinv2.embeddings.patch_embeddings.projection |    4.704K              |    18.874M |
|    swinv2.embeddings.norm                        |    0.192K              |    1.966M  |
|   swinv2.encoder.layers                          |   27.572M              |   6.661G   |
|    swinv2.encod

In [ ]:
# Print Original Model Results
print('---- Original Model Results ---- \n')
print(f'Accuracy: {orig_accuracy["accuracy"] * 100:.2f}% \n')
print(f'Precision: {orig_precision["precision"] * 100:.2f}% \n')
print(f'Recall: {orig_recall["recall"] * 100:.2f}% \n')
print(f'F1 Score: {orig_f1["f1"] * 100:.2f}% \n')
print(f'Average Inference Time per Batch: {orig_avg_inference_time:.6f} seconds \n')

print('Calculating NPs, FLOPs, MACs for single sample \n')
print(f'Number of Parameters (NPs): {org_total_params / 1e6:.2f} M \n')
print(f'FLOPs: {flop_count_table(org_flops)} \n')
print(f'MACs: {org_macs / 1e9:.2f} G\n')

---- Original Model Results ---- 

Accuracy: 81.74% 

Precision: 84.97% 

Recall: 81.74% 

F1 Score: 81.62% 

Average Inference Time per Batch: 0.031349 seconds 

Calculating NPs, FLOPs, MACs for single sample 

Number of Parameters (NPs): 28.35 M 

FLOPs: | module                                           | #parameters or shape   | #flops     |
|:-------------------------------------------------|:-----------------------|:-----------|
| model                                            | 28.347M                | 6.683G     |
|  swinv2                                          |  27.578M               |  6.682G    |
|   swinv2.embeddings                              |   4.896K               |   20.84M   |
|    swinv2.embeddings.patch_embeddings.projection |    4.704K              |    18.874M |
|    swinv2.embeddings.norm                        |    0.192K              |    1.966M  |
|   swinv2.encoder.layers                          |   27.572M              |   6.661G   |
|    swinv2.enc

In [ ]:
model.swinv2.layernorm.weight


Parameter containing:
tensor([0.0000, 0.0000, 3.9995, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 2.7550, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        3.1626, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 3.3621,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 2.9524, 2.6259, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.

In [ ]:
original_model.swinv2.layernorm.weight

Parameter containing:
tensor([ 1.2858e+00,  2.7892e+00,  3.9985e+00,  2.7193e+00,  2.3325e+00,
         3.6349e+00,  2.9811e+00,  3.2132e+00,  2.7926e+00,  2.2835e+00,
         2.7539e+00,  2.9471e+00,  2.9324e+00,  2.7301e+00,  2.7966e+00,
         2.9769e+00,  2.6193e+00,  3.0922e+00,  3.1618e+00,  2.3429e+00,
         2.9898e+00,  3.1319e+00,  3.0971e+00,  2.9987e+00,  2.8678e+00,
         2.6130e+00,  3.3611e+00,  2.8634e+00,  2.6790e+00,  2.7263e+00,
         2.2420e+00,  3.0492e+00,  2.3291e+00,  2.2954e+00,  2.6931e+00,
         2.5969e+00,  1.5440e+00,  2.5684e+00,  2.6792e+00,  2.6313e+00,
         3.1546e+00,  2.5841e+00,  2.5663e+00,  3.8745e+00,  2.7666e+00,
         3.3117e+00,  2.9660e+00,  2.8197e+00,  2.6077e+00,  3.9885e+00,
         3.0107e+00,  1.2215e+00,  2.2473e+00,  2.8262e+00,  2.3194e+00,
         2.5544e+00,  2.6423e+00,  3.2415e+00,  2.8065e+00,  2.9361e+00,
         2.9766e+00,  2.5896e+00,  2.2447e+00,  3.1194e+00,  2.6064e+00,
         3.0744e+00,  3.2981e

In [ ]:
model.swinv2.layernorm.bias

Parameter containing:
tensor([ 4.0045e-01,  7.4382e-02,  2.7739e-02,  7.3733e-02, -3.0586e-02,
        -1.2474e-02,  1.2447e-01,  9.1776e-02,  9.7027e-02,  1.7399e-01,
        -2.6889e-02,  1.1720e-01,  3.8335e-02, -3.4085e-03,  6.1383e-02,
         1.9356e-01,  3.3907e-02, -1.6321e-02,  4.8282e-02,  1.3613e-01,
         5.4141e-02,  9.3868e-02,  1.1907e-02,  1.9823e-01,  4.5181e-02,
         1.6518e-01,  8.3663e-02,  1.4254e-01,  8.2263e-02,  1.3887e-01,
         2.4370e-01,  1.2300e-01,  1.2716e-01, -6.4899e-03,  1.5414e-01,
         2.8738e-02, -3.5823e-02, -1.5177e-02,  1.7211e-01,  9.0239e-02,
        -1.3744e-02,  1.3784e-01, -1.1932e-02,  1.3948e-01,  1.8768e-01,
         3.9764e-02, -2.4103e-02, -2.6377e-02,  1.2630e-01,  1.4262e-01,
         2.0032e-01, -2.2233e-01,  3.9622e-01,  2.2318e-01,  2.8958e-01,
         2.1080e-01,  8.0193e-02,  1.5616e-01,  1.3643e-01,  1.4767e-01,
         4.1733e-02,  2.0241e-01,  4.4351e-03,  1.0778e-01,  1.1854e-01,
         1.0951e-01,  7.9630e